In [1]:
# Passos de execução

# Passo_1: Importar e tratar as Bases de Dados
# Passo_2: Criar 1 arquivo para cada loja  
# Passo_3: Salvar o backup nas pastas
# Passo_4: Calcular os indicadores
# Passo_5: Enviar o OnePage (e-mail padronizado em html)
# Passo_6: Enviar o email para a diretoria

# Importar as biblioteas

In [2]:
# importar bibliotecas
import pandas as pd
from pathlib import Path
import shutil
import win32com.client as win32


# Importar e tratar as Bases de Dados

In [3]:
#Passo_1: Importar e tratar as Bases de Dados

#Caminho
caminho= Path(r'C:\Users\fiu126\Desktop\Impressionador\Automações de Processo - Aplicação de Mercado de Trabalho\Projeto AutomacaoIndicadores')
#print (caminho)

#importar as  base dados
vendas_df = pd.read_excel(f"{caminho}\Bases de Dados\Vendas.xlsx")
lojas_df = pd.read_csv(f"{caminho}\Bases de Dados\Lojas.csv", encoding='latin-1',sep=';')
emails=pd.read_excel(f"{caminho}\Bases de Dados\Emails.xlsx")


# E-mails para os gerentes da loja

In [5]:
# Criar 1 arquivo para cada loja

for i,id_loja in enumerate(lojas_df['ID Loja']):
    
    # Passo_2: Criar 1 arquivo para cada loja  
    
    # guardo o nome da loja
    data_pedido= str(vendas_df['Data'].max()). replace('-','_')[0:10]
    nome_loja=lojas_df.loc[i,'Loja']
    
    # crio um dataframe apenas com as observações da loja
    vendas_loja_df=vendas_df[vendas_df['ID Loja']==id_loja]
    # coloco o nome da loja no dataframe e organizo as colunas
    df_loja_alterado=pd.merge(vendas_loja_df,lojas_df, on="ID Loja", how='left')
    df_loja_alterado=df_loja_alterado[['ID Loja','Loja', 'Código Venda','Data','Produto','Quantidade','Valor Unitário','Valor Final']]
    
    
    # Passo_3: Salvar o backup nas pastas
    #criar a pasta com o nome da loja se não existir
    if not Path(f"{caminho}\Backup Arquivos Lojas\{nome_loja}"). exists():
        Path(f"{caminho}\Backup Arquivos Lojas\{nome_loja}").mkdir()
        #print(f'A pasta {nome_loja} foi criada com sucesso.')
    #else:
        #print(f' A pasta {nome_loja} já existe.')
        
    # gravar o dataframe como backup:   
    df_loja_alterado.to_excel(f"{caminho}\Backup Arquivos Lojas\{nome_loja}\{data_pedido}.xlsx")  
    #print(f' Foi criado o ficheiro para a loja {nome_loja}.')
    
    # Passo_4: Calcular os indicadores
    #faturmaneto -> ano / dia
    faturamento_ano= df_loja_alterado['Valor Final'].sum()
    faturamento_diario=df_loja_alterado['Valor Final'][df_loja_alterado['Data']==vendas_df['Data'].max()].sum()
   
    #diversidade
    qtd_produtos_ano=len(df_loja_alterado['Produto'].unique())
    qtd_produtos_dia=len(df_loja_alterado['Produto'][df_loja_alterado['Data']==vendas_df['Data'].max()].unique())
    
    #ticket medio -> ano / dia
    valor_venda_ano=df_loja_alterado.groupby('Código Venda').sum()
    tiketmedio_ano=valor_venda_ano['Valor Final'].mean()
    valor_venda_dia=df_loja_alterado[df_loja_alterado['Data']==vendas_df['Data'].max()].groupby('Código Venda').sum()
    tiketmedio_dia=valor_venda_dia['Valor Final'].mean()
    
    #metas
    meta_faturamento_dia=1000
    meta_faturamento_ano=1650000
    meta_qtd_produtos_dia=4
    meta_qtd_produtos_ano=120
    meta_ticket_medio_dia=500
    meta_ticket_medio_ano=500
    
    # Passo_5: Enviar o OnePage
    
    outlook = win32.Dispatch('outlook.application')
    
    email=emails[emails['Loja']== nome_loja]['E-mail'].values[0]
    gerente=emails[emails['Loja']== nome_loja]['Gerente'].values[0]
    
    mail = outlook.CreateItem(0) 
    mail.To = email
    mail.Subject = f"OnePage Dia {data_pedido.replace('_', '-')} - Loja {nome_loja}"
    
    #mail.Body = f'''
    
    if faturamento_diario > meta_faturamento_dia:
        cor_fat_dia= 'green'
    else:
        cor_fat_dia= 'red'
    
    if faturamento_ano > meta_faturamento_ano:
        cor_fat_ano= 'green'
    else:
        cor_fat_ano= 'red'
        
    if qtd_produtos_dia > meta_qtd_produtos_dia:
        cor_qtde_dia= 'green'
    else:
        cor_qtde_dia= 'red'
        
    if qtd_produtos_ano > meta_qtd_produtos_ano:
        cor_qtde_ano= 'green'
    else:
        cor_qtde_ano= 'red'
    
    if tiketmedio_dia > meta_ticket_medio_dia:
        cor_ticket_dia= 'green'
    else:
        cor_ticket_dia= 'red'
  
    if tiketmedio_ano > meta_ticket_medio_ano:
        cor_ticket_ano= 'green'
    else:
        cor_ticket_ano= 'red'
    
    
    mail.HTMLBody = f'''
    <p>Bom dia, {gerente}</p>

    <p>O resultado de ontem <strong>({data_pedido})</strong> da <strong>Loja {nome_loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_diario:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtd_produtos_dia}</td>
        <td style="text-align: center">{meta_qtd_produtos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${tiketmedio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticket_medio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtd_produtos_ano}</td>
        <td style="text-align: center">{meta_qtd_produtos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${tiketmedio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticket_medio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Daniel</p>
    '''
    # Anexos (posso colocar quantos quiser):
    
    attachment  = f"{caminho}\Backup Arquivos Lojas\{nome_loja}\{data_pedido}.xlsx"
    mail.Attachments.Add(attachment)

    mail.Display(True) # -> apenas para ver o e-mail
    
    #mail.Send() ->enviar o e-mail (bloqueado)
    
    print(f"Email da loja {nome_loja} enviado com sucesso")
    
    #teste
    #print(f"O faturamento do dia {data_pedido.replace('_', '-')} da loja {nome_loja} foi de {faturamento_diario}.\nNo ano o faturamento totalizou {faturamento_ano}.")
    
    
    
    

Email da loja Iguatemi Esplanada enviado com sucesso
Email da loja Shopping Midway Mall enviado com sucesso
Email da loja Norte Shopping enviado com sucesso
Email da loja Shopping Iguatemi Fortaleza enviado com sucesso
Email da loja Shopping União de Osasco enviado com sucesso
Email da loja Shopping Center Interlagos enviado com sucesso
Email da loja Rio Mar Recife enviado com sucesso
Email da loja Salvador Shopping enviado com sucesso
Email da loja Rio Mar Shopping Fortaleza enviado com sucesso
Email da loja Shopping Center Leste Aricanduva enviado com sucesso
Email da loja Ribeirão Shopping enviado com sucesso
Email da loja Shopping Morumbi enviado com sucesso
Email da loja Parque Dom Pedro Shopping enviado com sucesso
Email da loja Bourbon Shopping SP enviado com sucesso
Email da loja Palladium Shopping Curitiba enviado com sucesso
Email da loja Passei das Águas Shopping enviado com sucesso
Email da loja Center Shopping Uberlândia enviado com sucesso
Email da loja Shopping Recife en

# E-mail para a diretoria

In [9]:
data_pedido= str(vendas_df['Data'].max()). replace('-','_')[0:10]

# Criar ranking para diretoria -> ano
faturamento_por_lojas=vendas_df.groupby('ID Loja').sum()
faturamento_lojas=lojas_df.merge(faturamento_por_lojas, on='ID Loja')[['Loja','Valor Final']]
faturamento_lojas_ano=faturamento_lojas.sort_values(by='Valor Final', ascending=False)

# Criar ranking para diretoria -> dia
vendas_dia=vendas_df.loc[vendas_df['Data']==vendas_df['Data'].max(),:]
faturamento_lojas_dias=vendas_dia.groupby('ID Loja').sum()
faturamento_lojas_dias=lojas_df.merge(faturamento_lojas_dias, on='ID Loja')[['Loja','Valor Final']]
faturamento_lojas_dias=faturamento_lojas_dias.sort_values(by='Valor Final', ascending=False)

display(faturamento_lojas_ano)
display(faturamento_lojas_dias)

# gravar o dataframe como backup:  
if not Path(f"{caminho}\Backup Arquivos Lojas\Diretoria"). exists():
        Path(f"{caminho}\Backup Arquivos Lojas\Diretoria").mkdir()
    
faturamento_lojas_dias.to_excel(r"{}\Backup Arquivos Lojas\Diretoria\ranking_dia_{}.xlsx".format(caminho,data_pedido), index=False)  
faturamento_lojas_ano.to_excel(r"{}\Backup Arquivos Lojas\Diretoria\ranking_ano_{}.xlsx".format(caminho,data_pedido), index=False) 



,Loja,Valor Final
23,Iguatemi Campinas,1762419
18,Shopping Vila Velha,1731167
13,Bourbon Shopping SP,1726110
6,Rio Mar Recife,1722766
19,Shopping SP Market,1721763
14,Palladium Shopping Curitiba,1721120
2,Norte Shopping,1711968
10,Ribeirão Shopping,1707122
0,Iguatemi Esplanada,1699681
8,Rio Mar Shopping Fortaleza,1698430


,Loja,Valor Final
4,Salvador Shopping,3950
15,Novo Shopping Ribeirão Preto,3400
10,Center Shopping Uberlândia,2651
13,Shopping Eldorado,2391
3,Shopping Center Interlagos,1582
11,Shopping Recife,1366
1,Norte Shopping,1259
2,Shopping União de Osasco,1207
12,Shopping Vila Velha,937
8,Bourbon Shopping SP,676


In [10]:
#enviar o e-mail
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {data_pedido}'

mail.Body = f"""

Prezados, bom dia

Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dias.iloc[0,0]} com Faturamento R${faturamento_lojas_dias.iloc[0,1]:.2f}
Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dias.iloc[-1,0]} com Faturamento R${faturamento_lojas_dias.iloc[-1,1]:.2f}

Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.iloc[0,0]} com Faturamento R${faturamento_lojas_ano.iloc[0, 1]:.2f}
Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.iloc[-1,0]} com Faturamento R${faturamento_lojas_ano.iloc[-1, 1]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Att.,

Daniel

"""

# Anexos (pode colocar quantos quiser):

attachment  = r'{}\Backup Arquivos Lojas\Diretoria\ranking_dia_{}.xlsx'.format(caminho,data_pedido)
mail.Attachments.Add(attachment)

attachment  = r'{}\Backup Arquivos Lojas\Diretoria\ranking_ano_{}.xlsx'.format(caminho,data_pedido)
mail.Attachments.Add(attachment)


mail.Display(True) # -> apenas para ver o e-mail
    
#mail.Send() ->enviar o e-mail (bloqueado)
print('E-mail da Diretoria enviado')



E-mail da Diretoria enviado


# Extra: código alternativo

In [11]:
#Criar um dicionário vazio e depois cada loja ser uma entrada desse dicionário 

# passo_1: colocar o nome da loja no dataset vendas (juntar pelo ID Loja)
df_vendas_alterado=pd.merge(vendas_df,lojas_df, on="ID Loja", how='left')

#passo_2: criar o dicionário
dicionario_lojas = {}
for loja in lojas_df['Loja']:
    dicionario_lojas[loja] = df_vendas_alterado.loc[df_vendas_alterado['Loja']==loja, :]


In [12]:
#ver se correu bem:
display(dicionario_lojas['Rio Mar Recife'])
#display(dicionario_lojas['Shopping Vila Velha'])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
62,46,2019-01-02,7,Camisa,1,100,100,Rio Mar Recife
63,46,2019-01-02,7,Calça Liso,2,190,380,Rio Mar Recife
64,46,2019-01-02,7,Cinto,1,200,200,Rio Mar Recife
113,87,2019-01-02,7,Camisa Estampa,1,113,113,Rio Mar Recife
142,109,2019-01-02,7,Camisa Gola V Listrado,3,116,348,Rio Mar Recife
...,...,...,...,...,...,...,...,...
100858,69905,2019-12-25,7,Mochila Listrado,2,275,550,Rio Mar Recife
100859,69905,2019-12-25,7,Camisa Xadrez,3,105,315,Rio Mar Recife
100860,69905,2019-12-25,7,Tênis Liso,3,266,798,Rio Mar Recife
100929,69954,2019-12-25,7,Chinelo,3,60,180,Rio Mar Recife
